In [ ]:
import os
from pathlib import Path
from path_utils import use_path

base_dir = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()

# Use relative paths from the base directory
config_path = str(base_dir / "configs/datasets/nuplan/8cams_undistorted.yaml")
checkpoint_path = str(base_dir / "output/master-project/run_omnire_undistorted_8cams_0")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyquaternion import Quaternion

# Function to get ego vehicle position from the ego pose matrix
def get_ego_position_info(ego_pose_matrix):
    # Extract position from the translation part of the matrix (last column)
    position = ego_pose_matrix[:3, 3]
    
    # Extract rotation as quaternion
    rotation_matrix = ego_pose_matrix[:3, :3]
    # Create quaternion from rotation matrix
    quat = Quaternion(matrix=rotation_matrix)
    # Extract yaw (heading) from quaternion as Euler angle
    yaw = quat.yaw_pitch_roll[0]
    
    return {
        "position": position,
        "x": position[0],
        "y": position[1],
        "z": position[2],
        "quaternion": quat,
        "heading": yaw,
        "heading_degrees": np.degrees(yaw)
    }

# Specify the scene directory and first frame index
def print_ego_starting_position(scene_dir, frame_idx=0):
    # Load the ego pose matrix for the first frame
    ego_pose_path = os.path.join(scene_dir, "ego_pose", f"{frame_idx:03d}.txt")
    
    if not os.path.exists(ego_pose_path):
        print(f"Error: Ego pose file not found at {ego_pose_path}")
        return None
    
    try:
        # Load the 4x4 transformation matrix
        ego_pose_matrix = np.loadtxt(ego_pose_path)
        
        # Get position and orientation information
        ego_info = get_ego_position_info(ego_pose_matrix)
        
        # Print the starting position information
        print("=== Ego Vehicle Starting Position (Frame 0) ===")
        print(f"Position (x, y, z): ({ego_info['x']:.3f}, {ego_info['y']:.3f}, {ego_info['z']:.3f})")
        print(f"Heading: {ego_info['heading']:.3f} rad, {ego_info['heading_degrees']:.2f}°")
        print(f"Quaternion (w, x, y, z): ({ego_info['quaternion'].w:.3f}, {ego_info['quaternion'].x:.3f}, {ego_info['quaternion'].y:.3f}, {ego_info['quaternion'].z:.3f})")
        print("Full transformation matrix:")
        print(ego_pose_matrix)
        
        # Return the information for further use if needed
        return ego_info
    except Exception as e:
        print(f"Error reading ego pose file: {e}")
        return None

In [ ]:
# List available scenes in the NuPlan dataset
import glob

# Path to the NuPlan processed data directory
nuplan_data_dir = os.path.join(base_dir, "drivestudio/data/nuplan/processed/mini")

# Check if the directory exists
if os.path.exists(nuplan_data_dir):
    # List all scene directories
    scene_dirs = glob.glob(os.path.join(nuplan_data_dir, "*"))
    scene_dirs = [d for d in scene_dirs if os.path.isdir(d)]
    
    print(f"Found {len(scene_dirs)} scenes in NuPlan dataset:")
    for i, scene_dir in enumerate(scene_dirs):
        print(f"{i+1}. {os.path.basename(scene_dir)}")
else:
    print(f"NuPlan data directory not found at {nuplan_data_dir}")
    print("Please update the path to point to your NuPlan processed data location.")

In [ ]:
# Analyze the ego position of the first scene (or specify a different scene index)
scene_index = 0  # Change this to analyze a different scene

if os.path.exists(nuplan_data_dir) and len(scene_dirs) > scene_index:
    selected_scene = scene_dirs[scene_index]
    print(f"Analyzing scene: {os.path.basename(selected_scene)}\n")
    
    # Print ego starting position for the selected scene
    ego_info = print_ego_starting_position(selected_scene)
    
    # Optionally, visualize the ego position in a 2D plot
    if ego_info:
        plt.figure(figsize=(8, 8))
        plt.scatter(ego_info['x'], ego_info['y'], color='red', s=100, marker='o', label='Ego Position')
        
        # Draw an arrow indicating the heading direction
        arrow_length = 5.0  # adjust as needed
        dx = arrow_length * np.cos(ego_info['heading'])
        dy = arrow_length * np.sin(ego_info['heading'])
        plt.arrow(ego_info['x'], ego_info['y'], dx, dy, head_width=1.0, head_length=1.5, fc='blue', ec='blue', label='Heading')
        
        plt.axis('equal')
        plt.grid(True)
        plt.xlabel('X (m)')
        plt.ylabel('Y (m)')
        plt.title('Ego Vehicle Starting Position')
        plt.legend()
        plt.show()
else:
    print("Cannot analyze ego position: No valid scene found.")